<a href="https://colab.research.google.com/github/nitinverma78/personal-health-insights-agent/blob/main/phia_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Clone the GitHub Repository

First, you need to clone the GitHub repository to your Colab environment. Replace `https://github.com/your-username/your-repo.git` with the actual URL of the repository you want to clone.

In [ ]:
# Replace with your GitHub repository URL
!git clone https://github.com/your-username/your-repo.git

# After cloning, you might want to list the contents to see the directory structure
!ls

### 2. Navigate into the Repository Directory

Once cloned, change your current working directory to the repository's root folder. Replace `your-repo` with the actual name of the cloned directory (usually the repository name).

In [ ]:
# Change directory into your cloned repository
%cd your-repo

# Verify the current working directory
!pwd
!ls

### 3. Install Dependencies

Most GitHub repositories have a `requirements.txt` file listing their Python dependencies. You can install them using `pip`. If your repository uses other package managers or has no `requirements.txt`, you'll need to adjust this step accordingly.

In [ ]:
# Install Python dependencies (if a requirements.txt file exists)
!pip install -r requirements.txt

# If you need to install specific packages not in requirements.txt, e.g.:
# !pip install pandas numpy matplotlib

### 4. Run Your Code

Now that the repository is cloned and dependencies are installed, you can run the main script, a Python notebook, or any other part of the repository. Replace `main_script.py` with the actual entry point of your code.

In [ ]:
# Example: Run a Python script
# !python main_script.py

# Example: If your repository contains Jupyter notebooks, you can open and run them in Colab directly after changing the directory.

# For example, to run a specific function or import modules from your repo:
# from your_repo_module import some_function
# result = some_function()
# print(result)

---

**Note:** You might encounter errors if your repository has specific system dependencies, requires environment variables, or expects certain data files not included in the repository. You'll need to address those on a case-by-case basis.

In [1]:
# Cell 1: Imports
# Note: the first run of this cell may take several minutes to complete

import os
import sys
import importlib
import glob
from IPython.display import HTML, display
import pandas as pd
import google.generativeai as genai
import ipywidgets

current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

from onetwo import ot
from onetwo.backends import gemini_api

# Import local modules
try:
    import data_utils
    import colab_utils
    import prompt_templates
    import phia_agent

    importlib.reload(data_utils)
    importlib.reload(colab_utils)
    importlib.reload(prompt_templates)
    importlib.reload(phia_agent)
    print("Modules imported/reloaded.")
except ImportError as e:
    print(f"Error importing local modules: {e}")
    print("Please ensure data_utils.py, colab_utils.py, prompt_templates.py, and phia_agent.py are in the same directory as the notebook.")

from data_utils import load_persona
from colab_utils import format_react_state_html
from phia_agent import get_react_agent, QUESTION_PREFIX

ModuleNotFoundError: No module named 'onetwo'

In [ ]:
# Cell 2: Set API Keys

# Replace with your API keys below.
#
# WARNING: Don't leave your API keys here when committing
# this notebook into any public code repos!
google_api_key = ""
tavily_api_key = ""

# Set the Google API key below and when setting up backend,
# the tavily API key will be set when creating the agent.
genai.configure(api_key=google_api_key)

In [ ]:
# Cell 3: Setup LLM Backend

llm_engine = gemini_api.GeminiAPI(
    generate_model_name="models/gemini-2.5-pro", # Can replace with a better model, if desired.
                                                 # Note: due to the nature of our few_shots
                                                 # examples that were designed with Gemini 1.5
                                                 # models in mind, agent behavior with Gemini
                                                 # 2.0, 2.5, 3.0 and beyond models may have errors
                                                 # related to output formatting. Brief testing with
                                                 # this notebook's new default (2.5 Pro) confirms
                                                 # that PHIA's functionality seems to be intact.
                                                 #
                                                 # All Gemini 1.5 models were deprecated alongside
                                                 # the release of Gemini 3.0 models.
    api_key=google_api_key,
    temperature=0.0,
)
llm_engine.register()
print("LLM Backend Registered.")

In [ ]:
# Cell 4: Load Data

summary_path = os.path.join("synthetic_wearable_users", "summary_df_502.csv")
activities_path = os.path.join("synthetic_wearable_users", "exercise_df_502.csv")

print(f"Loading data from: {summary_path} and {activities_path}")

try:
    if not os.path.exists(summary_path):
        raise FileNotFoundError(f"Cannot find summary file: {summary_path}")
    if not os.path.exists(activities_path):
        raise FileNotFoundError(f"Cannot find activities file: {activities_path}")

    summary_df, activities_df, profile_df = load_persona(
        summary_path=summary_path,
        activities_path=activities_path,
        enforce_schema=True,
        temporally_localize="today"
    )
    print("Data Loaded Successfully:")
    display(f"Summary DF: {summary_df.shape}")
    display(summary_df.head(1))
    display(f"Activities DF: {activities_df.shape}")
    display(activities_df.head(1))
    display(f"Profile DF: {profile_df.shape}")
    display(profile_df.head(1))
except Exception as e:
    print(f"Error loading data: {e}")

In [ ]:
# Cell 5: Load Exemplars

exemplar_dir = "few_shots"
exemplar_pattern = os.path.join(exemplar_dir, "*.ipynb")
final_exemplar_paths = glob.glob(exemplar_pattern)

if not final_exemplar_paths:
    print(f"Warning: No exemplar notebooks found in {exemplar_dir}/")
else:
    print(f"Found {len(final_exemplar_paths)} exemplar notebooks:")
    for path in final_exemplar_paths:
        print(f"  - {path}")

In [ ]:
# Cell 6: Create Agent

try:
    agent = get_react_agent(
        summary_df=summary_df,
        activities_df=activities_df,
        profile_df=profile_df,
        example_files=final_exemplar_paths,
        tavily_api_key=tavily_api_key,
        use_mock_search=False
    )
    print("Agent Created Successfully!")
except Exception as e:
    print(f"Error creating agent: {e}")

In [ ]:
# Cell 7: Ask a Question

# Define your question below
question = "How can I feel more awake and energetic during the day?"
full_question = QUESTION_PREFIX + question

try:
    final_answer, final_state = ot.run(
        agent(inputs=full_question, return_final_state=True)
    )

    print("\n--- Agent Trace ---")
    html_output = format_react_state_html(final_state)
    display(HTML(html_output))

    print("\n--- Final Answer ---")
    if final_answer:
        display(HTML(str(final_answer).replace('\n', '<br>')))
    else:
        display("No final answer was returned by the agent.")

except Exception as e:
    print(f"An error occurred during agent execution: {e}")